Before you turn this lab in, make sure everything runs as expected. **Restart and run all cells** (in the menubar, select Kernel$\rightarrow$Restart & Run All) then check the output is as expected and there are no errors.  Also remember to **Save** before uploading this file (File$\rightarrow$Save and Checkpoint).

---

# <div align="center">Quantum Computer Systems Design</div>&nbsp;  <div align="center">Lab 2</div> 

### 1. Quantum Measurements
How do we read out information from a quantum computer? In this problem, we will work with the different ways you may measure quantum states. 

#### 1.1 Measuring single-qubit states
Implement a quantum program that prepares a quantum state and then performs a measurement.

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute
from qiskit.tools.visualization import circuit_drawer, plot_bloch_multivector
from qiskit.quantum_info import state_fidelity, Operator
from qiskit import BasicAer
# Use the simulator from qiskit Aer
backend = BasicAer.get_backend('qasm_simulator')

In [ ]:
def draw_bloch(qc):
    state_simulator = BasicAer.get_backend('statevector_simulator')
    result = execute(qc, state_simulator).result()
    return plot_bloch_multivector(result.get_statevector(qc))

qc1_prep = QuantumCircuit(1, 1)
qc1_prep.ry(np.pi/8, 0) # prepare quantum state

draw_bloch(qc1_prep) # Visualize prepared state on Bloch sphere

In [ ]:
qc1 = QuantumCircuit(1, 1) 
qc1 += qc1_prep
qc1.measure(0, 0) # Z-basis measurement

qc1.draw(fold=-1) # Visualize quantum circuit

In [ ]:
job = execute(qc1, backend, shots=1000) # Simulate
job.result().get_counts(qc1)

##### 1.1.1 Z-basis measurement
Recall the definition of a z-basis measurement is to apply the $Z$ observable, written as:
$$Z = (+1)E_0 + (-1)E_1$$
where $E_0=|0\rangle\langle 0|$ and $E_1=|1\rangle\langle 1|$ are the orthogonal projectors associated with the z-basis.

What is the probability `pz` of obtaining $|0\rangle$ from the measurement?

In [ ]:
import numpy as np

# Define the state |psi\ = a|0\ + b|1\
alpha = 1/np.sqrt(2)
beta = 1/np.sqrt(2)
psi = np.array([alpha, beta])

# Define the projector E_0 = |0\/0|
E0 = np.array([[1, 0], [0, 0]])

# Compute the probability of measuring |0\
pz = np.vdot(psi, E0 @ psi)
print(pz)


0.4999999999999999


##### 1.1.2 Z-basis measurement (continued)
What is the expectation value of the above measurement, $\langle Z\rangle$?

In [ ]:
import numpy as np

# Define the state |psi\ = a|0\ + b|1\
alpha = 1/np.sqrt(2)
beta = 1/np.sqrt(2)
psi = np.array([alpha, beta])

# Define the Z observable
Z = np.array([[1, 0], [0, -1]])

# Compute the expectation value /Z\
expectation_Z = np.vdot(psi, Z @ psi)
print(expectation_Z)
# This will output 0.0, since the probabilities of measuring |0\rangle and |1\rangle are equal.


0.0


In [ ]:
# The autograder will test the value stored in `Z`
assert float(Z) == Z, 'The value should be a real number'

##### 1.1.3 X-basis measurement
Similarly the definition of an x-basis measurement is to apply the $X$ observable, written as:
$$X = (+1)E_+ + (-1)E_-$$
where $E_+=|+\rangle\langle +|$ and $E_-=|-\rangle\langle -|$ are the orthogonal projectors associated with the x-basis.

Sine Qiskit only support z-basis measurement, we need to create our own x-basis measurement. Use the fact that $HZH=X$, write down the circuit that accomplishes the x-basis measurment for the same quantum state from the last question:

In [ ]:
qc2_prep = QuantumCircuit(1, 1) # Initialize circuit with one qubit
qc2_prep.ry(np.pi/8, 0) # Prepare quantum state

draw_bloch(qc2_prep) # Visualize on Bloch sphere

In [ ]:
qc2 = QuantumCircuit(1, 1)
qc2 += qc2_prep

# X-basis measurement
from qiskit import QuantumCircuit, Aer, execute
qc = QuantumCircuit(1, 1)
# Prepare the same state as before
qc.h(0)
# Apply Hadamard before measurement to switch to x-basis
qc.h(0)
# Measure in z-basis (which now corresponds to x-basis)
qc.measure(0, 0)

# Simulate the circuit
backend = Aer.get_backend('qasm_simulator')
result = execute(qc, backend, shots=1024).result()
counts = result.get_counts()

print(counts)


qc2.draw(fold=-1)

In [ ]:
job = execute(qc2, backend, shots=1000) # Simulate
job.result().get_counts(qc2)

In [ ]:
_qc2_prep = QuantumCircuit(1, 1)
_qc2_prep.ry(np.pi/8, 0)
assert qc2_prep[:] == _qc2_prep[:], 'Do not modify the state preparation part'
assert qc2[:len(_qc2_prep)] == _qc2_prep[:], 'Do not modify the state preparation part'

_meas = QuantumCircuit(1, 1)
_meas.measure(0, 0)
assert qc2[-1:] == _meas[:], 'Your circuit must end with a measurement'

##### 1.1.4 X-basis measurement (continued)
What is the probability `pz` of obtaining $|+\rangle$ from the measurement?

In [ ]:
import numpy as np

# Define |+\ state
plus = np.array([1, 1]) / np.sqrt(2)
psi = plus
pz = np.abs(np.vdot(plus, psi))**2
print(pz)


0.9999999999999996


##### 1.1.5 X-basis measurement (continued)
What is the expectation value of the above measurement, $\langle X\rangle$?

In [ ]:
import numpy as np

# Define |+\ and |−\ states
plus = np.array([1, 1]) / np.sqrt(2)
minus = np.array([1, -1]) / np.sqrt(2)
X = np.outer(plus, plus) - np.outer(minus, minus)
psi = plus

expectation_X = np.vdot(psi, X @ psi)
print(expectation_X)
# This should giva an output near 1.0, confirming that the expectation value of measuring X in the |+\rangle state is 1.



0.9999999999999997


#### 1.2 Measuring multi-qubit states
Implement the quantum program that prepares the quantum state $|\psi\rangle=\frac{1}{\sqrt{3}}(|01\rangle+|10\rangle+|11\rangle)$.

In [ ]:
qc3_prep = QuantumCircuit(2, 1)
qc3_prep.ry(np.arccos(1/3**0.5)*2, 0)
qc3_prep.cry(np.pi/2, 0, 1)
qc3_prep.x(1)

qc3_prep.draw(fold=-1)

In [ ]:
# Compute the state psi
state_simulator = BasicAer.get_backend('statevector_simulator')
result = execute(qc3_prep, state_simulator).result()
psi = result.get_statevector(qc3_prep)
print(f'psi = {psi.round(10)!r}')

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector

# Create a quantum circuit with 2 qubits
qc = QuantumCircuit(2)

# Apply Ry and controlled gates to build the desired superposition
qc.ry(2 * np.arccos(1 / np.sqrt(3)), 0)  # Prepare amplitude on qubit 0
qc.x(1)                                  # Flip qubit 1 to |1\
qc.cx(0, 1)                              # Entangle qubit 0 and 1

state = Statevector.from_instruction(qc)
print(state)


Statevector([0.        +0.j, 0.81649658+0.j, 0.57735027+0.j,
             0.        +0.j],
            dims=(2, 2))


##### 1.2.1 Partial measurement
Say we want to measure observable $I\otimes X$.  What is the probability of measuring $|+\rangle$ on the second qubit?  You may calculate $p_{IX}$ by hand or compute it using the variable `psi` calculated above.

In [ ]:
import numpy as np

zero = np.array([1, 0])
one = np.array([0, 1])
plus = (zero + one) / np.sqrt(2)
psi = (np.kron(zero, one) + np.kron(one, zero) + np.kron(one, one)) / np.sqrt(3)

# Define E_+ = |+\/+| for second qubit
E_plus = np.outer(plus, plus)
# Identity on first qubit
I = np.eye(2)

# Observable: I direct product E_+
IX = np.kron(I, E_plus)

# Compute probability
p_ix = np.vdot(psi, IX @ psi)
print(p_ix)


0.8333333333333334


In [11]:
# The autograder will test the value stored in `p_ix`
assert 0 <= p_ix <= 1, 'The probability should be between 0 and 1'

##### 1.2.2 Partial measurement (continued)
What is the post-measurement state for the first qubit? Write down the density matrix representation of the quantum state. For example, if your answer is $\rho = \begin{pmatrix}1/2 & 0\\ 0 & 1/2\end{pmatrix}$, enter
```python
rho = np.array([[1/2, 0],
                [0, 1/2]])
```
You may calculate $\rho$ by hand or compute it using the variable `psi` calculated earlier.

In [12]:
import numpy as np

zero = np.array([1, 0])
one = np.array([0, 1])
plus = (zero + one) / np.sqrt(2)
psi = (np.kron(zero, one) + np.kron(one, zero) + np.kron(one, one)) / np.sqrt(3)
rho_full = np.outer(psi, psi)
E_plus = np.outer(plus, plus)
IX = np.kron(np.eye(2), E_plus)

# Apply projection
rho_proj = IX @ rho_full @ IX
# Normalize by probability p_{IX} = 5/6
rho_proj /= np.trace(rho_proj)

# Partial trace over second qubit to get reduced state of first qubit
rho_first = np.zeros((2, 2), dtype=complex)
for i in range(2):
    for j in range(2):
        rho_first[i, j] = rho_proj[i*2, j*2] + rho_proj[i*2+1, j*2+1]

rho = rho_first.real.round(6)  # Round for clarity
print(rho)


[[0.2 0.4]
 [0.4 0.8]]


In [13]:
# The autograder will test the value stored in `rho`
assert rho.shape == (2, 2), 'rho must be a 2 by 2 numpy array'
assert np.isclose(np.trace(rho), 1), 'Invalid density matrix: the trace must equal 1'

#### 1.3 A sequence of projective measurements

Implement a quantum program that prepares the boring quantum state $|\psi\rangle=|00\rangle$ using projective measurements.

In [ ]:
qc4_prep = QuantumCircuit(2) 
# No gates
qc4_prep.draw(fold=-1)

In [ ]:
# Compute the state psi
state_simulator = BasicAer.get_backend('statevector_simulator')
result = execute(qc4_prep, state_simulator).result()
psi4 = result.get_statevector(qc4_prep)
print(f'psi = {psi4.round(10)!r}')

In [ ]:
# The observable X direct sum X
o1_circ = QuantumCircuit(2)
o1_circ.x(0)
o1_circ.x(1)
o1_circ.draw(fold=-1)

In [ ]:
# The observable Z direct sum Z
o2_circ = QuantumCircuit(2)
o2_circ.z(0)
o2_circ.z(1)
o2_circ.draw(fold=-1)

In [ ]:
q = QuantumRegister(3, 'q')
c = ClassicalRegister(1, 'c')
qc4 = QuantumCircuit(q, c) # Initialize circuit with two qubits

# Prepare the initial state (qubits 1 and 2)
qc4.append(qc4_prep.to_gate(label='prep'), [1, 2])
qc4.barrier()

# Measure observable 1
qc4.h(0)
qc4.append(o1_circ.to_gate(label='O1').control(), [0, 1, 2])
qc4.h(0)
qc4.measure(0, c)
qc4.barrier()

qc4.x(0, label='reset').c_if(c, 1)  # Reset the measurement qubit to 0
qc4.z(2).c_if(c, 1)
qc4.barrier()

# Measure observable 2
qc4.h(0)
qc4.append(o2_circ.to_gate(label='O2').control(), [0, 1, 2])
qc4.h(0)
qc4.measure(0, c)
qc4.barrier()

qc4.x(0, label='reset').c_if(c, 1)  # Reset the measurement qubit to 0

qc4.draw(fold=-1)

##### 1.3.1 Projective measurement
What is the state vector `psi_after_o1_0` (for qubits 1 and 2) after measureing $O_1$, the observable $X\otimes X$ (at the 2nd barrier)?  Assume the outcome of the measurement is 0.

For example, if your answer is $\psi = \begin{pmatrix}1/2 \\ 1/2 \\ 1/2 \\ -1/2\end{pmatrix}$, enter
```python
psi_after_o1_0 = np.array([1, 1, 1, -1]) / 2
```

In [14]:
import numpy as np

# Define Pauli X
X = np.array([[0, 1], [1, 0]])
I = np.eye(2)
psi = (np.kron([1, 0], [0, 1]) + np.kron([0, 1], [1, 0]) + np.kron([0, 1], [0, 1])) / np.sqrt(3)

# Projector onto +1 eigenspace of X direct sum X
P_plus = 0.5 * (np.kron(I, I) + np.kron(X, X))

# Apply projection and normalize
psi_proj = P_plus @ psi
psi_after_o1_0 = psi_proj / np.linalg.norm(psi_proj)

print(np.round(psi_after_o1_0, 6))

[0.316228 0.632456 0.632456 0.316228]


In [15]:
# The autograder will test the value stored in `psi_after_o1_0`
assert psi_after_o1_0.shape == (4,), (
    'psi_after_o1_0 must be a 1D numpy array with 4 entries')
assert np.isclose(np.linalg.norm(psi_after_o1_0), 1), (
    'The state vector must be normalized')